In [ ]:
import pandas as pd 
import re 
from glob import glob
from pathlib import Path
import tabula
import pandas as pd


In [379]:
def return_keys_in_seq(filep):
    with open(filep,'r') as f:
        data = f.read()  
    pattern = re.compile(r'[^a-zA-Z]')
    key_seq = pattern.sub('', data)
    return key_seq

In [488]:
# Extract text from pdf 
from PyPDF2 import PdfReader
import io

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

pdf_text_gmat = extract_text_from_pdf('processed_pdf/ilovepdf_split-range/RC1-1-291.pdf')

In [493]:
pattern = re.compile(r'[^\x00-\x7F]')
clean_text = pattern.sub('', pdf_text_gmat)

In [ ]:
with open('clean_text.txt','w') as f:
    f.write(clean_text)
# corrected passage 17 


In [ ]:
# !!!!! manually step to remove space between "passage 1 7" and make it "passage 17" in clean_text.txt !!!! 

In [ ]:
# Read back text file 
with open('clean_text.txt','r') as f:
    clean_text = f.read()

In [506]:
pattern = r'(Passage\s+\d+\s*)\(\d+/\d+\)\s*[\r\n]+(.*?)(?=Passage\s+\d+\s*\(\d+/\d+\)|$)'
matches = re.findall(pattern, clean_text, re.DOTALL)

#### Ground Truth

In [ ]:
# Get the ground truth for each question 
# Path to your PDF file
pdf_path = 'processed_pdf/ilovepdf_split-range/RC1-992-1025.pdf'
# Read the tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
# Optionally, you can save the tables to a CSV file
for i, table in enumerate(tables):
    table.to_csv(f"processed_pdf/GT/table_{i+1}.csv", index=False)

files = glob('processed_pdf/GT/table_*.csv')
def natural_sort_key(filename):
    # Extract the numeric part of the filename using regex
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else filename
sorted_files  = sorted(files, key=natural_sort_key)
for idx, f in enumerate(sorted_files):
    oldpath = Path(f)
    old_name = oldpath.name
    new_name = f'table_{idx+1}.csv'
    new_file = oldpath.name.replace(old_name,new_name)
    new_path= oldpath.parent/"final"/new_file
    pd.read_csv(oldpath).to_csv(new_path,index=False)

In [ ]:
def natural_sort_key(filename):
    # Extract the numeric part of the filename using regex
    numbers = re.findall(r'\d+', filename)
    return int(numbers[0]) if numbers else filename


#### Structure the passages and questions/answers 

In [ ]:
# Build dict of question and choices
list_of_dicts = []
for idx, match in enumerate(matches):
    single_passage_dict = {
    "passage" : {
        "num": 0,
        "text": "",
        "Question":[]        

    }
}
    passage_no, content = match
    
    split_text = re.split(r'(?=\n\s*\d+\.\s)', content.strip())

    single_passage_dict["passage"]["num"] = passage_no
    single_passage_dict["passage"]["text"] = split_text[0].strip().\
        replace("\n","").\
            replace("GMAT","").\
            replace("GRE","").\
            replace("LSAT","")
    jdx = 0 
    for question in split_text[1:]:  
        qitem = {"item": 
                    {
                    "text": "",
                    "choices_text": [],
                    "correct_answer": ""
                }
            }
        choice_split = re.split(r'(?=\n\s*\([A-E]\))', question)
        
        q_text = choice_split[0].\
            strip().\
            replace("\n","").\
            replace("GMAT","").\
            replace("GRE","").\
            replace("LSAT","") 
        qpattern = re.compile(r'^\d+')
        q_match = qpattern.search(q_text)
        # print(f"{idx+1}-{jdx}")
        if int(q_match.group()) >1000:
            # print("XX",q_match.group())
            continue 
        qitem["item"]["text"] = q_text
        # get key file name 
        #key_filepath = sorted_files[idx]
        #key_seq = return_keys_in_seq(key_filepath)
        #print(f"filepath:{key_filepath}")
        #qitem["item"]["correct_answer"] = key_seq[jdx]
        jdx+=1
        for c in choice_split[1:]:
            # remove \n from choices_split
            qitem["item"]["choices_text"].append(c.strip().\
                                                 replace("\n","").\
                                                replace("GMAT","").\
                                                replace("GRE","").\
                                                replace("LSAT",""))
        single_passage_dict["passage"]["Question"].append(qitem)
    list_of_dicts.append(single_passage_dict)

In [513]:
import json 
with open('passage.json','w') as f:
    json.dump(list_of_dicts,f)

In [514]:
len(list_of_dicts)

112

In [ ]:
# Map correct answers to the Question 
with open('passage.json','r') as f:
    passage_dict = json.load(f)
files = glob('processed_pdf/GT/final/table_*.csv')
sorted_files  = sorted(files, key=natural_sort_key)

for idx , p in enumerate(passage_dict):
    # print(p)
    key_filepath = sorted_files[idx]
    key_seq = return_keys_in_seq(key_filepath)
    # print(p['passage']['num'])
    # print(key_filepath)
    # print(key_seq)
    #print(key_seq)
    for jdx , q in enumerate(p['passage']['Question']):
        # print(f"{idx}-{jdx}")
        q['item']['answer']=key_seq[jdx]

In [518]:
import json 
with open('passage_with_keys.json','w') as f:
    json.dump(passage_dict,f)